In [82]:
#@formatter:off
%load_ext autoreload
%autoreload 2
#@formatter:on

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
import pandas as pd
from mbench import (
    Parameter,
    ParameterData,
    CountryData,
    AdjacentDistricts,
    Country,
    District,
    Districts,
    OldDistrictToNewDistrict,
    demographic,
    util,
)


In [84]:
if sys.platform == 'linux':
    os.chdir('/mnt/d/Dropbox/benchmarking')
elif sys.platform =='Darwin':
    os.chdir('/Users/sepmein/Dropbox/benchmarking')

data_dir = os.path.abspath('Data')
openmalaria_xml_config_path = os.path.abspath('Data/openmalaria/12936_2017_2051_MOESM1_ESM.xml')
openmalaria_xml_config_path_pn2 = os.path.abspath("Data/openmalaria/VanDucAP2.xml")
openmalaria_xml_config_path_pn3 = os.path.abspath("Data/openmalaria/VanDucAP3.xml")

In [85]:
ahafo = District(iso='gh-af', aliases=['AHAFO'])
ashanti = District(iso='gh-ah', aliases=['ashanti'])
bono = District(iso='gh-bo', aliases=['bono'])
bono_east = District(iso='gh-be', aliases=['bono_east'])
central = District(iso='gh-cp', aliases=['central'])
eastern = District(iso='gh-ep', aliases=['eastern'])
greater_accra = District(iso='gh-aa', aliases=['greater_accra'])
north_east = District(iso='gh-ne', aliases=['north_east'])
northern = District(iso='gh-np', aliases=['northern'])
oti = District(iso='gh-ot', aliases=['oti'])
savannah = District(iso='gh-sv', aliases=['savannah'])
upper_east = District(iso='gh-ue', aliases=['upper_east'])
upper_west = District(iso='gh-uw', aliases=['upper_west'])
volta = District(iso='gh-tv', aliases=['volta'])
western = District(iso='gh-wp', aliases=['western'])
western_north = District(iso='gh-wn', aliases=['western_north'])

gha_districts = Districts(district_list=[
    ahafo, ashanti, bono, bono_east, central, eastern, greater_accra, north_east, northern, oti, savannah, upper_east, upper_west, volta, western, western_north
])
gha_districts.n

16

In [86]:
district_216_260 = pd.read_csv(data_dir + "/GHA/ADM1/216 to 260.csv")

In [87]:
gha_old_new_districts = OldDistrictToNewDistrict(
    start = district_216_260['216'],
    to= district_216_260['260'],
    districts = gha_districts,
    columns = ['old','new']
)
gha_old_new_districts.data

,old,new
0,BRONG_AHAFO,GH_AF
1,ASHANTI,GH_AH
2,BRONG_AHAFO,GH_BO
3,BRONG_AHAFO,GH_BE
4,CENTRAL,GH_CP
5,EASTERN,GH_EP
6,GREATER_ACCRA,GH_AA
7,NORTHERN,GH_NP
8,NORTHERN,GH_NE
9,VOLTA,GH_OT


In [88]:
# load adjacent
gha_adjacent_provinces_data = pd.read_excel(
    data_dir + "/gha_adm_adjacent.xlsx", sheet_name="adm1_adjacent"
)
gha_adjacent_provinces_data = demographic.reformat.adm1_name(
    df=gha_adjacent_provinces_data,
    original_column_name="a",
    new_column_name="from",
    set_index=False,
)
gha_adjacent_provinces_data = demographic.reformat.adm1_name(
    df=gha_adjacent_provinces_data,
    original_column_name="b",
    new_column_name="to",
    set_index=False,
)

gha_adjacent_provinces = AdjacentDistricts(
    start= gha_adjacent_provinces_data['from'],
    to = gha_adjacent_provinces_data['to'],
    districts = gha_districts
)

In [89]:
gha = Country(districts=gha_districts,
              adjacent_districts=gha_adjacent_provinces,
              old_districts_to_new_districts = gha_old_new_districts
              )
gha

In [90]:
gha_data = CountryData(country = gha)

In [91]:
gha_data.index

Index(['GH_AF', 'GH_AH', 'GH_BO', 'GH_BE', 'GH_CP', 'GH_EP', 'GH_AA', 'GH_NE',
       'GH_NP', 'GH_OT', 'GH_SV', 'GH_UE', 'GH_UW', 'GH_TV', 'GH_WP', 'GH_WN'],
      dtype='object')

In [92]:
gha_adm1_vector_resistance = pd.read_excel(
    data_dir
    + "/GHA/WHO_IR_data/MTM_DISCRIMINATING_CONCENTRATION_BIOASSAY_20211130.xlsx",
    sheet_name="Data",
)
gha_adm1_vector_resistance = gha_adm1_vector_resistance.groupby("ADMIN1").mean()
gha_adm1_vector_resistance = gha_adm1_vector_resistance["MORTALITY_ADJUSTED"]
gha_adm1_vector_resistance

ADMIN1
Ashanti            51.592958
Brong Ahafo        42.100000
Central            40.071053
Eastern            32.664865
Greater Accra      32.057600
Kassena Nankana    81.725000
Northern           72.059341
Upper East         53.837500
Upper West         44.305882
Volta              52.340000
Western            64.860000
Name: MORTALITY_ADJUSTED, dtype: float64

In [93]:
vector_resistance = Parameter(
    name= 'vector resistance',
    aliases= ['vr', 'vector_resistance']
)
pm_gha_vector_resistance = ParameterData(
    parameter= vector_resistance,
    value=gha_adm1_vector_resistance,
    country=gha,
    from_old_districts_system= True
)
pm_gha_vector_resistance._value

,old,new,MORTALITY_ADJUSTED
0,BRONG_AHAFO,GH_AF,NaN
1,ASHANTI,GH_AH,NaN
2,BRONG_AHAFO,GH_BO,NaN
3,BRONG_AHAFO,GH_BE,NaN
4,CENTRAL,GH_CP,NaN
5,EASTERN,GH_EP,NaN
6,GREATER_ACCRA,GH_AA,NaN
7,NORTHERN,GH_NP,NaN
8,NORTHERN,GH_NE,NaN
9,VOLTA,GH_OT,NaN


In [94]:
gha_districts.reformat(to_format=)

SyntaxError: invalid syntax (1349196799.py, line 1)